In [1]:
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
gtzan_dir = 'GTZAN/genres_original/'

In [3]:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


In [4]:
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'rolloff': None, 'contrast': None, 'bandwidth': None, 'flatness': None,
                'flux': None, 'rmse': None, 'zcr': None, 'chroma':None }
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
    features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr)
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr)
    features['flatness'] = librosa.feature.spectral_flatness(y)
    features['zcr'] = librosa.feature.zero_crossing_rate(y)
    features['rmse'] = librosa.feature.rms(y)
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr)
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr)
 
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y,n_mfcc=21)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [5]:
def read_process_songs(src_dir, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
        
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name, mono=True, duration=30)
                
                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))
                
                # Append the result to the data structure
                features = get_features(signal, sr)
                features['genre'] = x
                arr_features.append(features)
    return arr_features


In [6]:
%%time

# Get list of dicts with features and convert to dataframe
features = read_process_songs(gtzan_dir, debug=False)

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:7: FutureWarning: Pass y=[0.00732422 0.01660156 0.00762939 ... 0.18188477 0.1637268  0.15093994] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:8: FutureWarning: Pass y=[0.00732422 0.01660156 0.00762939 ... 0.18188477 0.1637268  0.15093994] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:9: FutureWarning: Pass y=[0.00732422 0.01660156 0.00762939 ... 0.18188477 0.1637268  0.15093994] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
C:\Users\high-\AppData\L

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:9: FutureWarning: Pass y=[-0.01300049 -0.03060913 -0.03607178 ... -0.00515747 -0.00244141
  0.00283813] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:10: FutureWarning: Pass y=[-0.01300049 -0.03060913 -0.03607178 ... -0.00515747 -0.00244141
  0.00283813] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:11: FutureWarning: Pass y=[-0.01300049 -0.03060913 -0.03607178 ... -0.00515747 -0.00244141
  0.00283813] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['flatness'] = librosa.feature.spectral_flatness(y)
C:\U

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:11: FutureWarning: Pass y=[ 0.1822815   0.28302002  0.21020508 ... -0.02590942  0.07104492
  0.15505981] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['flatness'] = librosa.feature.spectral_flatness(y)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:13: FutureWarning: Pass y=[ 0.1822815   0.28302002  0.21020508 ... -0.02590942  0.07104492
  0.15505981] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rmse'] = librosa.feature.rms(y)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:19: FutureWarning: Pass y=[ 0.1822815   0.28302002  0.21020508 ... -0.02590942  0.07104492
  0.15505981] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y,n_mfcc=21)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:19: FutureWarning: Pass y=[ 0.04647827  0.06399536  0.04327393 ... -0.00680542 -0.0177002
 -0.02374268] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y,n_mfcc=21)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:34: FutureWarning: Pass y=[ 0.04647827  0.06399536  0.04327393 ... -0.00680542 -0.0177002
 -0.02374268] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:7: FutureWarning: Pass y=[ 0.12014771  0.14956665  0.10458374 ... -0.06948853 -0.07192993
 -0.07434082] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
C:\Users\high-\AppData\Local\Temp

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:7: FutureWarning: Pass y=[-0.13864136 -0.20974731 -0.1781311  ... -0.02752686 -0.02734375
 -0.03271484] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:8: FutureWarning: Pass y=[-0.13864136 -0.20974731 -0.1781311  ... -0.02752686 -0.02734375
 -0.03271484] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:9: FutureWarning: Pass y=[-0.13864136 -0.20974731 -0.1781311  ... -0.02752686 -0.02734375
 -0.03271484] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
C:\

...

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:10: FutureWarning: Pass y=[-0.02352905 -0.03363037 -0.03527832 ...  0.04922485  0.05480957
  0.0475769 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:11: FutureWarning: Pass y=[-0.02352905 -0.03363037 -0.03527832 ...  0.04922485  0.05480957
  0.0475769 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['flatness'] = librosa.feature.spectral_flatness(y)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:13: FutureWarning: Pass y=[-0.02352905 -0.03363037 -0.03527832 ...  0.04922485  0.05480957
  0.0475769 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rmse'] = librosa.feature.rms(y)
C:\Users\high-\AppData\Local

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:19: FutureWarning: Pass y=[0.05926514 0.06332397 0.03329468 ... 0.10351562 0.09851074 0.08352661] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y,n_mfcc=21)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:34: FutureWarning: Pass y=[0.05926514 0.06332397 0.03329468 ... 0.10351562 0.09851074 0.08352661] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:7: FutureWarning: Pass y=[-0.03121948 -0.06448364 -0.05792236 ...  0.1265564   0.13085938
  0.13464355] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:7: FutureWarning: Pass y=[ 0.07272339  0.10369873  0.10211182 ... -0.02728271 -0.03768921
 -0.01303101] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:8: FutureWarning: Pass y=[ 0.07272339  0.10369873  0.10211182 ... -0.02728271 -0.03768921
 -0.01303101] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:9: FutureWarning: Pass y=[ 0.07272339  0.10369873  0.10211182 ... -0.02728271 -0.03768921
 -0.01303101] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
C:\

C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:10: FutureWarning: Pass y=[-0.02111816 -0.03451538 -0.03536987 ... -0.02139282 -0.0050354
  0.02505493] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:11: FutureWarning: Pass y=[-0.02111816 -0.03451538 -0.03536987 ... -0.02139282 -0.0050354
  0.02505493] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['flatness'] = librosa.feature.spectral_flatness(y)
C:\Users\high-\AppData\Local\Temp/ipykernel_3920/3440509940.py:13: FutureWarning: Pass y=[-0.02111816 -0.03451538 -0.03536987 ... -0.02139282 -0.0050354
  0.02505493] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rmse'] = librosa.feature.rms(y)
C:\Users\high-\AppData\Local\Te

Wall time: 12min 18s


In [7]:
df_features = pd.DataFrame(features)

In [8]:
df_features.head()

,centroid_max,centroid_min,centroid_mean,centroid_std,rolloff_max,rolloff_min,rolloff_mean,rolloff_std,contrast_max,contrast_min,...,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_20_max,mfcc_20_min,mfcc_20_mean,mfcc_20_std,tempo,genre
0,4031.364538,1027.079258,1784.416546,360.194166,7644.287109,1830.322266,3806.418650,949.383832,48.289165,4.028252,...,32.866341,-23.219631,1.219947,6.853878,19.209179,-20.962769,-3.538389,6.161644,123.046875,blues
1,5444.293577,421.091729,1529.871314,613.197888,8085.717773,333.764648,3548.986873,1725.338349,48.775996,5.534489,...,27.740530,-17.131062,0.532144,6.769286,17.467901,-32.333900,-4.090439,6.538787,107.666016,blues
2,4669.770713,417.254802,1552.637786,395.649207,6556.860352,441.430664,3041.089944,884.842654,67.980056,4.558865,...,16.524359,-29.785425,-2.231391,5.534097,23.665482,-23.899199,-4.049148,6.599841,161.499023,blues
3,3236.442116,318.713500,1070.110059,429.542379,7611.987305,387.597656,2185.061787,1222.372310,53.952526,5.438508,...,16.595015,-21.426252,-3.406940,5.655832,29.324226,-18.845253,-1.500599,5.899070,172.265625,blues
4,4464.156642,849.658285,1835.507009,585.947299,7084.423828,1152.026367,3581.003346,1253.847208,54.526649,6.113804,...,4.889853,-41.730530,-11.701833,7.430688,20.845287,-34.022064,-5.859701,8.214063,135.999178,blues


In [9]:
df_features.to_csv('gtzan_features.csv', index=False)